# Lesson: Serving Your Chatbot with a RESTful API Using Flask
Serving Your Chatbot with a RESTful API Using Flask

Welcome to the next step in our journey of building a chatbot service with Flask. In the previous lesson, we focused on the ChatController, which manages chat sessions and handles messages by interacting with both the model and service layers. Now, we will take a significant step forward by creating a RESTful API for our chatbot service using Flask. We'll start by setting up the main Flask application, then adapt the ChatController to integrate with Flask's session management.
Understanding RESTful APIs and Flask

RESTful APIs are a way for different software systems to talk to each other over the internet. Think of them as a set of rules that allow programs to communicate and share data. Flask is a lightweight web framework that makes it easy to build these APIs. It's like a toolkit that helps you create web applications quickly and efficiently.

To get started with Flask, you can install it using pip by running the following command in your terminal or command prompt:

In [ ]:
pip install Flask

Now, we can use it to connect the components we've already built, allowing users to interact with our chatbot service through a web interface. This will enable seamless communication between users and our chatbot service.

First, we need to initialize the Flask application, load environment variables, and set a secret key for session management.

In [ ]:
from flask import Flask
from controllers.chat_controller import ChatController

# Initialize the Flask application
app = Flask(__name__)

# Set a secret key for session management
app.secret_key = 'your_secret_key_here'

# Create an instance of ChatController to handle chat operations
chat_controller = ChatController()

Here's a breakdown of the steps involved in this initialization process:

    Initialize the Flask Application: We start by importing the necessary modules, including Flask itself. The Flask class is then instantiated to create our application object, app. This object will be used to configure and run our web application.

    Set a Secret Key: Setting a secret key is crucial for session management in Flask. The secret key is used to cryptographically sign the session cookies, ensuring that the data stored in the session is secure and cannot be tampered with by clients. You should use a strong, random value for the secret key to enhance security.

    Create an Instance of the Controller: Finally, we create an instance of our ChatController. This controller will handle the core operations of our chatbot service, such as managing chat sessions and processing messages.

Defining Routes for the Chatbot Service

Next, we define the routes for our chatbot service. These routes will handle requests for creating a new chat session and sending messages.

In [ ]:
# Define a route for the index page that ensures a user session
@app.route('/')
def index():
    chat_controller.ensure_user_session()
    return "Welcome to the Chatbot Service!"

# Define a route for creating a new chat session
@app.route('/api/create_chat', methods=['POST'])
def create_chat():
    # Delegate the creation of a chat session to the chat controller
    return chat_controller.create_chat()

# Define a route for sending a message in an existing chat session
@app.route('/api/send_message', methods=['POST'])
def send_message():
    # Delegate the handling of a message to the chat controller
    return chat_controller.send_message()

Here's how each route functions within our chatbot service:

    Index Route (/): This route acts as the entry point to our chatbot service. When a user accesses this route, it ensures that a user session is established by invoking the ensure_user_session method of the ChatController. After setting up the session, it returns a welcome message to the user, indicating that the service is ready for interaction.

    Create Chat Route (/api/create_chat): This route is designed to initiate a new chat session. It listens for POST requests and delegates the task of creating a chat session to the ChatController by calling its create_chat method. The controller handles the session creation and returns a response, which is then sent back to the client.

    Send Message Route (/api/send_message): This route facilitates sending messages within an existing chat session. It also listens for POST requests and delegates the message processing to the ChatController by invoking its send_message method. The controller processes the message and returns the AI's response, which is then delivered to the client.

Running the Flask Server

Finally, we set up the Flask application to run on a specified port when executed directly.

In [ ]:
# Run the Flask application if this script is executed directly
if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=3000)

The if __name__ == '__main__': construct ensures that the Flask application runs only when the script is executed directly, rather than being imported as a module in another script. This is a common Python idiom for making code both importable as a module and executable as a standalone script. The app.run() method is then called to start the Flask development server with the following parameters:

    debug=True: Enables debug mode, which provides helpful error messages and automatic reloading of the server when code changes are detected.
    host='0.0.0.0': Makes the server accessible externally by listening on all available network interfaces, allowing connections from other devices on the network.
    port=3000: Specifies the port on which the server will listen for incoming requests, allowing users to access the application via http://localhost:3000.

Adapting the ChatController

To integrate the ChatController with Flask, we need to update it to use Flask's session management. We start by importing necessary Flask modules and updating the constructor to remove the test session.

In [ ]:
import uuid
from flask import session, request
from services.chat_service import ChatService

class ChatController:
    def __init__(self):
        self.chat_service = ChatService()

Next, let's break down how to update the ChatController methods to utilize Flask's session management.
ChatController: Ensuring User Session

In the ensure_user_session method, we update the way user sessions are managed by leveraging Flask's built-in session object. Previously, we used a test session dictionary to store user IDs. Now, we check if a user_id exists in the Flask session. If it doesn't, we generate a new UUID and store it in the session.

In [ ]:
def ensure_user_session(self):
    """Ensure user has a session ID."""
    # Check if 'user_id' is already in the session
    if 'user_id' not in session:
        # Generate a new UUID and store it in the session if not present
        session['user_id'] = str(uuid.uuid4())
    # Return the user_id from the session
    return session['user_id']

This change allows us to utilize Flask's session management capabilities, providing a more robust and scalable solution for maintaining user sessions.
ChatController: Creating a Chat

In the create_chat method, we update the code to now retrieve the user_id directly from the Flask session, ensuring that the request is associated with a valid user session.

In [ ]:
def create_chat(self):
    """Handle chat creation request."""
    # Retrieve the user_id from the session
    user_id = session.get('user_id')

    # Check if the session has expired (user_id is not present)
    if not user_id:
        return {'error': 'Session expired'}, 401
    
    # Create a new chat session using the ChatService
    chat_id = self.chat_service.create_chat(user_id)

    # Return a JSON response with the chat ID and success message
    return {
        'chat_id': chat_id,
        'message': 'Chat created successfully'
    }

If the session is expired, it returns an error response. Otherwise, it creates a new chat session using the ChatService and returns a JSON response with the chat ID and a success message.
ChatController: Sending a Message

The send_message method is updated to handle requests using Flask's request and session objects. We now extract chat_id and user_message from the incoming request using the request object.

In [ ]:
def send_message(self):
    """Handle message sending request."""
    # Retrieve the user_id from the session
    user_id = session.get('user_id')

    # Check if the session has expired (user_id is not present)
    if not user_id:
        return {'error': 'Session expired'}, 401
    
    # Extract chat_id and user_message from the incoming JSON request
    chat_id = request.json.get('chat_id')
    user_message = request.json.get('message')
    
    # Check if chat_id or user_message is missing
    if not chat_id or not user_message:
        return {'error': 'Missing chat_id or message'}, 400
        
    try:
        # Process the message using the ChatService and get the AI's response
        ai_response = self.chat_service.process_message(user_id, chat_id, user_message)
        # Return the AI's response as a JSON object
        return {'message': ai_response}
    except ValueError as e:
        # Handle specific error scenarios and return appropriate JSON responses
        return {'error': str(e)}, 404
    except RuntimeError as e:
        return {'error': str(e)}, 500

If any required data is missing, the method returns an error response. The message is processed using the ChatService, and the AI's response is returned as a JSON object. We've also implemented error handling to provide appropriate JSON responses for different scenarios, ensuring that all interactions with the API are consistent and informative.
How Clients Interact with the Chatbot API

To interact with our chatbot service, a client can follow these steps to send a message using the API:

    Access the Index Route (/): The client begins by accessing the index route of the API. This step ensures that a user session is established. A user session is a way to keep track of the user's interactions with the service. It is stored using Flask's session management, which utilizes cookies to maintain session data on the client side. The server responds with a welcome message, indicating that the service is ready for interaction.

    Create a Chat Session (/api/create_chat): Before sending a message, the client needs to create a new chat session. This is done by sending a request to the /api/create_chat route. The server will respond with a unique chat ID, which is essential for identifying the chat session in subsequent interactions. The user session ensures that the chat session is associated with the correct user, allowing for personalized interactions.

    Send a Message (/api/send_message): With the chat session established, the client can now send a message to the chatbot. This involves sending a request to the /api/send_message route, including the chat ID and the message content. The server processes the message and responds with the AI's reply, allowing the client to continue the conversation. The user session helps maintain continuity in the conversation by linking the messages to the correct user and chat session.

By following these steps, a client can effectively communicate with the chatbot service, leveraging the RESTful API to manage chat sessions and exchange messages while utilizing user sessions for a seamless experience.
Summary and Next Steps

In this lesson, we successfully built a RESTful API for our chatbot service using Flask. We set up a Flask application, defined routes for chat operations and updated the ChatController to utilize Flask's session management. This lesson marks a significant milestone in our course, as it brings together all the components we've developed so far into a functional web application.

As you move on to the practice exercises, take the opportunity to experiment with the Flask API and reinforce the concepts covered in this lesson. This hands-on practice will prepare you for further development and exploration of additional Flask features and RESTful API concepts. Keep up the great work, and I look forward to seeing your progress!

# exercises

You've made great progress in understanding Flask basics. Now, let's put that knowledge into practice by creating a very simple Flask application. Your task is to:

    Initialize a Flask app.
    Define an index route that returns a welcome message.
    Set up the app to run on port 3000 and host '0.0.0.0'.

This exercise is designed to help you build a basic Flask application that simply returns a message. At this stage, we are not involving any other layers of our app. Dive in and see how straightforward it can be!

In [ ]:
from flask import Flask

# TODO: Initialize the Flask application
app = Flask(__name__)

# TODO: Define a route for the index page
# The route should return "Welcome to the Chatbot Service!"
@app.route('/')
def index():
    return "Welcome to the Chatbot Service!"

# TODO: Run the Flask application if this script is executed directly
# Use host='0.0.0.0' and port=3000
if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=3000)

Having successfully built a basic Flask application, it's time to enhance its functionality by implementing session management within the ChatController. This task will guide you through integrating Flask's session capabilities to ensure efficient user session handling. Follow these steps to complete the task:

    Update main.py:
        Import the ChatController to manage chat operations.
        Set a secret key for session management to secure session data.
        Create an instance of ChatController to handle chat operations.
        Ensure a user session is established by calling the ensure_user_session method from the chat_controller.

    Modify /chat_controller.py:
        Import session from Flask to manage session data.
        Remove the test_session dictionary used for simulating sessions.
        Update the ensure_user_session method to check and store the user_id in Flask's session.
        Retrieve the user_id from Flask's session in the create_chat and send_message methods to ensure session validity.

This task will deepen your understanding of integrating Flask's session management into a RESTful API. Let's get started!

In [ ]:
import uuid
from services.chat_service import ChatService
# TODO: Import session from flask
from flask import session

class ChatController:
    def __init__(self):
        self.chat_service = ChatService()
        # self.test_session = {}  # TODO: Remove the dictionary used to simulate sessions
    
    def ensure_user_session(self):
        """Ensure user has a session ID in the test session."""
        # TODO: Check if 'user_id' is in Flask's session
        if 'user_id' not in session: #self.test_session:
            # TODO: Sotre new UUID in Flask's session
            session['user_id'] = str(uuid.uuid4()) #self.test_session['user_id'] = str(uuid.uuid4())
        # TODO: Return the user_id from Flask's session
        return session['user_id'] #self.test_session['user_id']
    
    def create_chat(self):
        """Handle chat creation request."""
        # TODO: Retrieve user_id from Flask's session
        user_id = session.get('user_id') #self.test_session.get('user_id')
        if not user_id:
            return {'error': 'Session expired'}, 401
        
        chat_id = self.chat_service.create_chat(user_id)
        return {
            'chat_id': chat_id,
            'message': 'Chat created successfully'
        }
    
    def send_message(self, chat_id, user_message):
        """Handle message sending request."""
        # TODO: Retrieve user_id from Flask's session
        user_id = session.get('user_id')
        if not user_id:
            return {'error': 'Session expired'}, 401
        
        if not chat_id or not user_message:
            return {'error': 'Missing chat_id or message'}, 400
            
        try:
            ai_response = self.chat_service.process_message(user_id, chat_id, user_message)
            return {'message': ai_response}
        except ValueError as e:
            return {'error': str(e)}, 404
        except RuntimeError as e:
            return {'error': str(e)}, 500

In [ ]:
from flask import Flask
# TODO: Import the ChatController
from controllers.chat_controller import ChatController

# Initialize the Flask application
app = Flask(__name__)

# TODO: Set a secret key for session management
app.secret_key = 'your_secret_key_here'
# TODO: Create an instance of ChatController to handle chat operations
chat_controller = ChatController()

# Define a route for the index page
@app.route('/')
def index():
    # TODO: Call chat_controller.ensure_user_session() to ensure session
    chat_controller.ensure_user_session()
    return "Welcome to the Chatbot Service!"

# Run the Flask application directly
if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=3000)

In this task, you'll take a significant step by creating a route that handles chat session creation.

Your goal is to add a new endpoint to your Flask application that creates a chat session when a client makes a request. Specifically, you need to:

    Create a route that listens for POST requests at the /api/create_chat endpoint
    Use the existing chat_controller to handle the chat creation logic
    Return the response from the controller back to the client

This route is essential for your chatbot service because it allows users to start new conversations. Before users can exchange messages with your chatbot, they first need to create a chat session, which is exactly what this endpoint will enable.

Your Flask app is already set up with an index route that welcomes users and ensures they have a session. Now it's time to build the functionality that lets them create chat sessions!

In [ ]:
from flask import Flask
from controllers.chat_controller import ChatController

# Initialize the Flask application
app = Flask(__name__)

# Set a secret key for session management
app.secret_key = 'your_secret_key_here'

# Create an instance of ChatController to handle chat operations
chat_controller = ChatController()

# Define a route for the index page that ensures a user session
@app.route('/')
def index():
    chat_controller.ensure_user_session()
    return "Welcome to the Chatbot Service!"

# TODO: Define a route for creating a new chat session
@app.route('/api/create_chat', methods=['POST'])
# - Listen for POST requests at '/api/create_chat'
def create_chat():
# - Delegate session creation to ChatController's create_chat
    return chat_controller.create_chat()
# - Return the response from the controller

# Run the Flask application if this script is executed directly
if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=3000)

In [ ]:
import uuid
from flask import session
from services.chat_service import ChatService

class ChatController:
    def __init__(self):
        self.chat_service = ChatService()
    
    def ensure_user_session(self):
        """Ensure user has a session ID."""
        if 'user_id' not in session:
            session['user_id'] = str(uuid.uuid4())
        return session['user_id']
    
    def create_chat(self):
        """Handle chat creation request."""
        user_id = session.get('user_id')
        if not user_id:
            return {'error': 'Session expired'}, 401
        
        chat_id = self.chat_service.create_chat(user_id)
        return {
            'chat_id': chat_id,
            'message': 'Chat created successfully'
        }
    
    def send_message(self, chat_id, user_message):
        """Handle message sending request."""
        user_id = session.get('user_id')
        if not user_id:
            return {'error': 'Session expired'}, 401
        
        if not chat_id or not user_message:
            return {'error': 'Missing chat_id or message'}, 400
            
        try:
            ai_response = self.chat_service.process_message(user_id, chat_id, user_message)
            return {'message': ai_response}
        except ValueError as e:
            return {'error': str(e)}, 404
        except RuntimeError as e:
            return {'error': str(e)}, 500

Great job on setting up the basic structure of your Flask application! Now, let's take an exciting step forward by enabling the ability to send messages to our chat via the API. This will allow us to interact with our chatbot service in real-time.

Here's what you'll be doing:

    Define a new route at the /api/send_message endpoint to handle POST requests for sending messages.
    Use the ChatController to manage the message-sending logic.
    Update the ChatController to extract chat_id and user_message from the request body using Flask's request object.

This will enhance your understanding of building RESTful APIs with Flask and bring your chatbot service to life. Dive in and see how these components come together to create a seamless user experience!

In [ ]:
# main.py
from flask import Flask
from controllers.chat_controller import ChatController

# Initialize the Flask application
app = Flask(__name__)

# Set a secret key for session management
app.secret_key = 'your_secret_key_here'

# Create an instance of ChatController to handle chat operations
chat_controller = ChatController()

# Define a route for the index page that ensures a user session
@app.route('/')
def index():
    chat_controller.ensure_user_session()
    return "Welcome to the Chatbot Service!"

# Define a route for creating a new chat session
@app.route('/api/create_chat', methods=['POST'])
def create_chat():
    return chat_controller.create_chat()
    
# TODO: Define a route for sending a message in an existing chat session
@app.route('/api/send_message', methods=['POST'])
# - Listen for POST requests at '/api/send_message'
def send_message():
# - Delegate message handling to ChatController's send_message
    return chat_controller.send_message()
# - Return the response from the controller

# Run the Flask application if this script is executed directly
if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=3000)

In [ ]:
# controllers.chat_controller.py
import uuid
from flask import session, request  # TODO: Import the 'request' object from Flask to handle incoming request data
from services.chat_service import ChatService

class ChatController:
    def __init__(self):
        self.chat_service = ChatService()
    
    def ensure_user_session(self):
        """Ensure user has a session ID."""
        if 'user_id' not in session:
            session['user_id'] = str(uuid.uuid4())
        return session['user_id']
    
    def create_chat(self):
        """Handle chat creation request."""
        user_id = session.get('user_id')
        if not user_id:
            return {'error': 'Session expired'}, 401
        
        chat_id = self.chat_service.create_chat(user_id)
        return {
            'chat_id': chat_id,
            'message': 'Chat created successfully'
        }
    
    # TODO: Remove the parameters 'chat_id' and 'user_message' from the method signaturerequest body
    def send_message(self):
        """Handle message sending request."""
        user_id = session.get('user_id')
        if not user_id:
            return {'error': 'Session expired'}, 401
        
        # TODO: Use the 'request' object to get 'chat_id' and 'user_message' from the JSON payload
        chat_id = request.json.get('chat_id')
        user_message = request.json.get('message')

        if not chat_id or not user_message:
            return {'error': 'Missing chat_id or message'}, 400
            
        try:
            ai_response = self.chat_service.process_message(user_id, chat_id, user_message)
            return {'message': ai_response}
        except ValueError as e:
            return {'error': str(e)}, 404
        except RuntimeError as e:
            return {'error': str(e)}, 500